In [ ]:
import requests, csv, os, re
from bs4 import BeautifulSoup
url = 'https://www.joytalentagency.com/results/models-women-mainboard/women'

In [ ]:
r =requests.get(url)
soup = BeautifulSoup(r.text, 'html5lib')

In [ ]:
links = soup.find('div', {"class":"results-grid"}).findAll('a')
names = soup.findAll('div',{'class':'name'})
model_names = [name.text.strip().replace('\n',' ') for name in names]
model_links = [link['href'] for link in links]
models_dict =  dict(zip(model_names,model_links))
models_details=list()

In [ ]:
models_dict

In [ ]:
for k,v in models_dict.items():
    os.makedirs(f"joytalentagency.com/{k}",exist_ok=True)
    soup1 = BeautifulSoup(requests.get(v).content,'html5lib')
    stats =soup1.find('div',{'class':'stats none'})
    feature = [feat.text for feat in stats.findAll('span',{'class':'stat-label'})]
    value = [val.text for val in stats.findAll('span',{'class':'value'})]

    stats_dict = dict(zip(feature,value))
    stats_dict['NAME'] = k
    stats_dict['URL'] = v
    images = soup1.find('div',{'class':'portfolio-page'}).findAll('a',{'class':'custom-fancybox-slider'})
    patt=r"\d+.jpg"
    print(len([i for i in images]))
    for i in images:
        img_link=i['href']
        print(img_link)
        filename=re.search(patt,img_link).group(0)
        path = f"joytalentagency.com/{k}/{filename}"
        if not os.path.exists(path):
            with open(path,'wb') as f:
                f.write(requests.get(img_link).content)
    models_details.append(stats_dict)
print(models_details[0])
print(len(models_details))

with open(f"./joytalentagency.com/models_info.csv",'w') as f:
    c = csv.DictWriter(f, ['NAME','URL','HEIGHT','BUST','WAIST','HIPS','DRESS','SHOE','HAIR','EYES'])
    c.writeheader()
    for row in models_details:
        c.writerow(row)